This Jupyter notebook contains a series of code cells that demonstrate the process of loading a pre-trained Densenet121 model, preprocessing a sample image, using the model to predict the class of the image, converting the model to TFLite format, and testing the TFLite model with the sample image. Additionally, it includes code for downloading and preparing an image from a URL for model inference.

In [2]:
import numpy as np

import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.11.1'

In [3]:
model = keras.models.load_model('../model/densenet121_v1_34_0.592.h5')

In [ ]:
from tensorflow.keras.preprocessing import image
# Load the image
img_path = '../data/clean-cifar-100-python/test-deployment/image_1.jpg' 
img = image.load_img(img_path, target_size=(75, 75))
# Preprocess the image
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255.  # the model was trained on inputs that were normalized in the range [0, 1]

# Use the model to predict the class of the image
prediction = model.predict(img_tensor)

In [4]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.densenet import preprocess_input

# Load the image
img_path = '../data/clean-cifar-100-python/test-deployment/image_1.jpg' 
img = image.load_img(img_path, target_size=(75, 75))

# Preprocess the image
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_tensor = preprocess_input(img_array)

# Use the model to predict the class of the image
prediction = model.predict(img_tensor)

# Add label names to the prediction probabilities
class_names = ['aquatic_mammals', 'fish', 'flowers', 'food_containers', 'fruit_and_vegetables', 'household_electrical_devices', 
               'household_furniture', 'insects', 'large_carnivores', 'large_man-made_outdoor_things',
               'large_natural_outdoor_scenes', 'large_omnivores_and_herbivores', 'medium_mammals', 'non-insect_invertebrates', 
               'people', 'reptiles', 'small_mammals', 'trees', 'vehicles_1', 'vehicles_2']

1/1 [==============================] - 2s 2s/step


In [5]:
dict(zip(class_names, prediction[0]))

{'aquatic_mammals': -6.5015044,
 'fish': -20.108147,
 'flowers': -1.783258,
 'food_containers': -2.350761,
 'fruit_and_vegetables': -1.9533087,
 'household_electrical_devices': -8.131875,
 'household_furniture': -8.4431,
 'insects': 2.8202362,
 'large_carnivores': 13.564195,
 'large_man-made_outdoor_things': -15.193861,
 'large_natural_outdoor_scenes': -9.958817,
 'large_omnivores_and_herbivores': 4.116876,
 'medium_mammals': 3.5384219,
 'non-insect_invertebrates': 0.19212708,
 'people': -14.425937,
 'reptiles': -12.128974,
 'small_mammals': 9.810526,
 'trees': -7.0088897,
 'vehicles_1': -11.2750635,
 'vehicles_2': -11.919313}

In [6]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('cifar-100-model.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\Ahmad\AppData\Local\Temp\tmpdzin7btf\assets


INFO:tensorflow:Assets written to: C:\Users\Ahmad\AppData\Local\Temp\tmpdzin7btf\assets


In [7]:
import os
os.path.getsize("cifar-100-model.tflite") / (1024*1024)

30.595596313476562

In [8]:
import tensorflow.lite as tflite
interpreter = tflite.Interpreter(model_path='cifar-100-model.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']
output_index

625

In [9]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [10]:
url = 'https://i.imgur.com/23MiAxv.jpg'
img = download_image(url)
target_size = (75, 75)
prep_img = prepare_image(img, target_size)

In [11]:
type(prep_img)

PIL.Image.Image

In [12]:
pix = prep_img.load()

# Get the RGB values of the first pixel
r, g, b = pix[0, 0]

print(f"The R value of the first pixel is {r}")

The R value of the first pixel is 6


In [13]:
normalized_r = r / 255
print(normalized_r)

0.023529411764705882


In [14]:
import requests
from PIL import Image
from io import BytesIO

# URL of the image
url = 'https://i.imgur.com/23MiAxv.jpg'

'''
# Send a HTTP request to the URL of the image
response = requests.get(url)

# Open the URL image as a PIL image object
img = Image.open(BytesIO(response.content))
'''

# Download the image from a url
img = download_image(url)

# Preprocess the image
target_size = (75, 75)
img = prepare_image(img, target_size) # Resize the image
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor = preprocess_input(img_tensor)

# Now use the .tflite model
interpreter = tflite.Interpreter(model_path='cifar-100-model.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

interpreter.set_tensor(input_index, img_tensor)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

# Print the prediction
dict(zip(class_names, preds[0]))

{'aquatic_mammals': -22.021074,
 'fish': -33.166653,
 'flowers': -15.633723,
 'food_containers': -6.470115,
 'fruit_and_vegetables': -9.440056,
 'household_electrical_devices': -2.7160254,
 'household_furniture': 1.8643621,
 'insects': 29.163816,
 'large_carnivores': 33.332577,
 'large_man-made_outdoor_things': -20.783209,
 'large_natural_outdoor_scenes': -21.852894,
 'large_omnivores_and_herbivores': -17.849588,
 'medium_mammals': 4.1026273,
 'non-insect_invertebrates': -1.5337784,
 'people': -9.650715,
 'reptiles': -20.151258,
 'small_mammals': -8.165364,
 'trees': -47.12472,
 'vehicles_1': -13.878261,
 'vehicles_2': 4.4217033}

To build the docker image for hw#09 Q6:

```bash
docker build -t ml-zoom-09 .
```

To run the docker container for hw#09 Q6:

```bash
docker run -p 9000:8080 -it ml-zoom-09
```

To test the lambda function in the docker container locally:

```bash
$ curl -XPOST "http://localhost:9000/2015-03-31/functions/function/invocations" -d '{"url":"https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg"}'
```

Output:

```json
{"Bee": 0.5546649396419525, "Wasp": 0.4453350603580475}
```